In [1]:
import timeit
import pandas as pd
import os
import re
import urllib.request
import bs4

ROOTDIR = os.path.normpath("N:\_archive\\test\\trans\soundgasmNET")
df = pd.read_csv(os.path.join(ROOTDIR, "sgasm_rip_db.csv"), sep=";", encoding="utf-8", index_col=0)
df_grp = df.groupby("sgasm_user")

url = "https://soundgasm.net/u/belle_in_the_woods/F4M-The-Take-FdomJOIcountdownteasingwhispers"

# Testing timeits of filtering downloaded urls

In [3]:
%timeit url in df_grp.get_group("belle_in_the_woods")["URLsg"]

The slowest run took 7.66 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 527 µs per loop


In [4]:
%timeit url in df[df["sgasm_user"] == "belle_in_the_woods"]["URLsg"]

100 loops, best of 3: 2.24 ms per loop


In [5]:
def t():
    df.query("sgasm_user == 'belle_in_the_woods'")["URLsg"].isin([url]).any()

In [6]:
%timeit t()

100 loops, best of 3: 2.65 ms per loop


In [7]:
%timeit url in df["URLsg"]

The slowest run took 39.70 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 9.47 µs per loop


In [26]:
s_for = ["https://soundgasm.net/u/belle_in_the_woods/F4M-The-Take-FdomJOIcountdownteasingwhispers",
       "https://soundgasm.net/u/belle_in_the_woods/F4F-Should-I-Call-intimateerotic-poetry",
       "https://soundgasm.net/u/belle_in_the_woods/F4M-In-Darkness-FsubErotic-Poetry",
       "https://soundgasm.net/u/belle_in_the_woods/F4M-Cum-Lust-cock-worshiperotic-poetry",
       "https://soundgasm.net/u/belle_in_the_woods/F4M-The-Take-FdomJOIcountdownteasingwhispers",
        "https://soundgasm.net/u/belle_in_the_woods/F4A-Ill-Be-Home-For-Christmas-singinga-capellaChristmas-song",
         "https://soundgasm.net/u/belle_in_the_woods/F4M-An-Early-Morning-Quickie-GFEvanillakissingmuffled-orgasmgigglesa-little-dirty-talkshort-but-sweet",
         "https://soundgasm.net/u/belle_in_the_woods/F4M-Turning-Fs-into-As-into-Fsublegal-teenteacherstudentmasturbationblowjobspankingroughanalchokingback-to-school-challenge",
         "https://soundgasm.net/u/belle_in_the_woods/F4M-Cum-For-Your-Good-Girl-Fsub-a-little-switching-light-Fdomgreedy-for-youbeggingdirty-talkwhisperspossible-ASMRcum-for-merelaxingsoundscape",
         "http:///spungfsgs.netz/UNIQUE/aknfksd"]

def filter_alrdy_downloaded(df, dl_dict, currentusr=None):
    # OLD when passing 2pair tuples, unpack tuples in dl_list into two lists
    # url_list, title = zip(*dl_list)
    # filter dupes
    unique_urls = set(dl_dict)
    if currentusr:
        try:
            duplicate = unique_urls.intersection(df_grp.get_group(currentusr)["URLsg"].values)
        except KeyError:
            logger.info("User '{}' not yet in databas!".format(currentusr))
            duplicate = set()
    else:
        # timeit 1000: 0.19
        duplicate = unique_urls.intersection(df["URLsg"].values)

    # set.symmetric_difference()
    # Return a new set with elements in either the set or other but not both.
    # -> duplicates will get removed from unique_urls
    result = list(unique_urls.symmetric_difference(duplicate))

    return result

In [13]:
%timeit filter_alrdy_downloaded(df, s_for)

The slowest run took 4.58 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 700 µs per loop


In [14]:
%timeit filter_alrdy_downloaded(df, s_for, currentusr="belle_in_the_woods")

The slowest run took 303.96 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 501 µs per loop


In [2]:
import sqlite3



conn = sqlite3.connect("N:\\_archive\\test/trans\\soundgasmNET\\gwarip_db.sqlite")
c = conn.cursor()
c.execute("SELECT url_sg FROM Downloads")
url_tupes = c.fetchall()
urls = [tupe[0] for tupe in url_tupes]
urls_set = set(urls)
conn.close()

def filter_alrdy_downloaded(df, urls, dled, currentusr=None):
    # OLD when passing 2pair tuples, unpack tuples in dl_list into two lists
    # url_list, title = zip(*dl_list)
    # filter dupes
    unique_urls = set(urls)
    # using grped df is slightly faster than checking all urls:
    # df: 1000 loops, best of 3: 705 µs per loop; grped_df: 1000 loops, best of 3: 488 µs per loop
    if currentusr:
        try:
            c.execute("SELECT url_sg FROM Downloads WHERE sgasm_user = ?", (currentusr,))
            usr_urls = [tup[0] for tup in c.fetchall()]
            duplicate = unique_urls.intersection(usr_urls)
        except KeyError:
            print("error")
            duplicate = set()
    else:
        # timeit 1000: 0.19
        duplicate = unique_urls.intersection(dled)

    dup_titles = "\n".join(duplicate)

    # set.symmetric_difference()
    # Return a new set with elements in either the set or other but not both.
    # -> duplicates will get removed from unique_urls
    result = list(unique_urls.symmetric_difference(duplicate))
    # str.contains accepts regex patter, join url strings with | -> htt..m4a|htt...m4a etc
    # returns Series/array of boolean values, .any() True if any element is True
    # timeit 1000: 1.129
    # mask = df["URL"].str.contains('|'.join(url_list))
    # isin also works
    # timeit 1000: 0.29
    # mask = df["URL"].isin(unique_urls)
    # print(df["URL"][mask])

    return result

In [29]:
%timeit urls_set = set(urls)

The slowest run took 6.72 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 449 µs per loop


In [17]:
%timeit filter_alrdy_downloaded(0, s_for, urls)

1000 loops, best of 3: 304 µs per loop


### Using a set instead of a list to hold the downloaded urls is ~80 times faster!!
And we only need to do the conversion to a set (that almost takes as long as searching for the url in the list) once at startup

In [18]:
%timeit filter_alrdy_downloaded(0, s_for, urls_set)

100000 loops, best of 3: 7.47 µs per loop


In [3]:
%timeit filter_alrdy_downloaded(0, s_for, urls_set, currentusr="belle_in_the_woods")
conn.close()

NameError: name 's_for' is not defined

In [19]:
conn = sqlite3.connect("N:\\_archive\\test/trans\\soundgasmNET\\gwarip_db.sqlite")
c = conn.cursor()

def filter_dled_sql(c, url):
    # the second argument we pass to replace ? must be a tuple -> () + , at the end -> tuple
    c.execute("SELECT url_sg FROM Downloads WHERE url_sg = ?", (url,))
    res = c.fetchone()
    if res:
        return True
    else:
        return False
    
    
def filter_urls_sql(c, urls):
    new_dl = []
    for url in urls:
        if not filter_dled_sql(c, url):
            new_dl.append(url)
    return new_dl

### Testing them one after the other instead of all at once is SO MUCH SLOWER

In [20]:
%timeit filter_urls_sql(c, s_for)

1 loop, best of 3: 217 ms per loop


## Testing sqlite functionality

In [12]:
try:
    os.remove("test_res/t-temp.db")
except FileNotFoundError:
    pass
conn = sqlite3.connect("test_res/t-temp.db")
# we can call execute on conn directly (it creates cursor obj and returns it)
c = conn.execute("CREATE TABLE test (id INT, name TEXT)")
c.execute("INSERT INTO test VALUES (2, 'milton')")
# we didnt call commit so no changes should be commited 
# If you don’t call this method, anything you did since the last call to commit() is not visible from other database connections.
# but they would be readable from the same conn
c.execute("SELECT * FROM test")
print("same conn: ", c.fetchall())

conn2 = sqlite3.connect("test_res/t-temp.db")
c2 = conn2.execute("SELECT * FROM test")
print("other conn: ", c2.fetchall())

# executescript calls to commit first then execs script
c.executescript("INSERT INTO test VALUES (3, 'max')")

c.execute("SELECT * FROM test")
print("same conn: ", c.fetchall())

conn2 = sqlite3.connect("test_res/t-temp.db")
c2 = conn2.execute("SELECT * FROM test")
print("other conn: ", c2.fetchall())

conn.close()
conn2.close()

same conn:  [(2, 'milton')]
other conn:  []
same conn:  [(2, 'milton'), (3, 'max')]
other conn:  [(2, 'milton'), (3, 'max')]


In [95]:
try:
    os.remove("test_res/t-temp.db")
except FileNotFoundError:
    pass
con = sqlite3.connect("test_res/t-temp.db")
con.execute("create table person (id integer primary key, firstname varchar unique)")

# Successful, con.commit() is called automatically afterwards
with con:
    con.execute("insert into person(firstname) values (?)", ("Joe",))

con2 = sqlite3.connect("test_res/t-temp.db")
c2 = con2.execute("SELECT * FROM person")
print(c2.fetchall())

c = con.execute("INSERT INTO person(firstname) VALUES ('does this get rolled back')")

# con.rollback() is called after the with block finishes with an exception, the
# exception is still raised and must be caught
try:
    with con:
        con.execute("insert into person(firstname) values (?)", ("Joe",))
except sqlite3.IntegrityError:
    print("couldn't add Joe twice")
c.execute("SELECT * FROM person")
print("with: ", c.fetchall())
   
c = con.execute("INSERT INTO person(firstname) VALUES ('does this get rolled back')")    

# without with
try:
    con.execute("insert into person(firstname) values (?)", ("Joe",))  # <- 1-tuple
except sqlite3.IntegrityError:
    print("couldn't add Joe twice")
c.execute("SELECT * FROM person")
print("without with: ", c.fetchall())


# with DOESNT CLOSE CONNECTION!!!!
con.close()
con2.close()

[(1, 'Joe')]
couldn't add Joe twice
with:  [(1, 'Joe')]
couldn't add Joe twice
without with:  [(1, 'Joe'), (2, 'does this get rolled back')]


In [106]:
import sqlite3
def clean_name(some_var):
    return ''.join(char for char in some_var if char.isalnum())

def set_missing_values_df_sql(cursor, audiodl_obj):
    # parameters with ? etc. only work with values not table/col names
    cursor.execute("SELECT * FROM Downloads WHERE url = ?", (audiodl_obj.url_to_file,))
#     row_cont = cursor.fetchone()
#     print(row_cont)
    # executescript doesnt work with parameters
    # use format instead and put ' around the inserted vals -> UNSAFE sql injections
    cursor.executescript("""
    UPDATE Downloads 
    SET url_sg = CASE WHEN url_sg IS NULL THEN '{}' ELSE url_sg END, 
        local_filename = CASE WHEN local_filename IS NULL THEN '{}' ELSE local_filename END WHERE url = '{}'
    """.format(audiodl_obj.page_url, audiodl_obj.filename_local, audiodl_obj.url_to_file))
    

def set_missing_values_df(cursor, audiodl_obj):
    # parameters with ? etc. only work with values not table/col names
    cursor.execute("SELECT * FROM Downloads WHERE url = ?", (audiodl_obj.url_to_file,))
    # get row
    row_cont = cursor.fetchone()
    # get indexes of None elements
    null_ind = (i for i, e in enumerate(row_cont) if e is None)
    # get table cols
    c.execute("PRAGMA table_info('Downloads')")
    col_name = [colinfo[1] for colinfo in c.fetchall()]
    # get names of col where val is None
    null_cols = [col_name[i] for i in null_ind]
    
    set_helper = (("reddit_title", "title"), ("reddit_url", "permalink"), ("reddit_user", "r_user"),
                       ("created_utc", "created_utc"), ("reddit_id", "id"), ("subreddit_name", "subreddit"),
                       ("r_post_url", "r_post_url"))

    
    upd_cols = []
    upd_vals = []
    if "url_sg" in null_cols:
        upd_cols.append("url_sg = ?")
        upd_vals.append(audiodl_obj.page_url)
    if "local_filename" in null_cols:
        upd_cols.append("local_filename = ?")
        upd_vals.append(audiodl_obj.filename_local)
    if audiodl_obj.reddit_info:
        for col, key in set_helper:
            if col in null_cols:
                upd_cols.append("{} = ?".format(col))
                upd_vals.append(audiodl_obj.reddit_info[key])
    if upd_cols:
        # append url since upd_vals need to include all the param substitutions for ?
        upd_vals.append(audiodl_obj.url_to_file)
        print(null_cols)
        # would work in SQLite version 3.15.0 (2016-10-14), but this is 3.8.11, users would have to update as well so not a good idea
        # print("UPDATE Downloads SET ({}) = ({}) WHERE url = ?".format(",".join(upd_cols), ",".join("?"*len(upd_cols))))
        
        # join only inserts the string to join on inbetween the elements of the iterable (none at the end)
        # format to -> e.g UPDATE Downloads SET url_sg = ?,local_filename = ? WHERE url = ?
        c.execute("UPDATE Downloads SET {} WHERE url = ?".format(",".join(upd_cols)), upd_vals)
    cursor.connection.commit()
    
    
def set_missing_values_df_sqlrow(cursor, audiodl_obj):
    # Row provides both index-based and case-insensitive name-based access to columns with almost no memory overhead
    cursor.connection.row_factory = sqlite3.Row
    # we need to create new cursor after changing row_factory
    new_c = cursor.connection.cursor()
    # reset row_factory to default so we get normal tuples when fetching (should we generate a new cursor)
    # new_c will always fetch Row obj and cursor will fetch tuples
    cursor.connection.row_factory = None
    new_c.execute("SELECT * FROM Downloads WHERE url = ?", (audiodl_obj.url_to_file,))
    # get row
    row_cont = new_c.fetchone()
    
    set_helper = (("reddit_title", "title"), ("reddit_url", "permalink"), ("reddit_user", "r_user"),
                       ("created_utc", "created_utc"), ("reddit_id", "id"), ("subreddit_name", "subreddit"),
                       ("r_post_url", "r_post_url"))

    
    upd_cols = []
    upd_vals = []
    if row_cont["url_sg"] is None:
        upd_cols.append("url_sg = ?")
        upd_vals.append(audiodl_obj.page_url)
    if row_cont["local_filename"] is None:
        upd_cols.append("local_filename = ?")
        upd_vals.append(audiodl_obj.filename_local)
    if audiodl_obj.reddit_info:
        for col, key in set_helper:
            if row_cont[col] is None:
                upd_cols.append("{} = ?".format(col))
                upd_vals.append(audiodl_obj.reddit_info[key])
    if upd_cols:
        # append url since upd_vals need to include all the param substitutions for ?
        upd_vals.append(audiodl_obj.url_to_file)
        # would work in SQLite version 3.15.0 (2016-10-14), but this is 3.8.11, users would have to update as well so not a good idea
        # print("UPDATE Downloads SET ({}) = ({}) WHERE url = ?".format(",".join(upd_cols), ",".join("?"*len(upd_cols))))
        
        # join only inserts the string to join on inbetween the elements of the iterable (none at the end)
        # format to -> e.g UPDATE Downloads SET url_sg = ?,local_filename = ? WHERE url = ?
        c.execute("UPDATE Downloads SET {} WHERE url = ?".format(",".join(upd_cols)), upd_vals)
    cursor.connection.commit()

In [109]:
import sys
# append pardir to syspath so we can import gwaripper (not the one in site-packages)
sys.path.append('../')
import gwaripper.gwaripper as gw

try:
    conn = sqlite3.connect("N:\\_archive\\test/trans\\soundgasmNET\\_dev\\gwarip_db.sqlite")
    c = conn.cursor()
    a = gw.AudioDownload("https://soundgasm.net/u/Testing1231234/testest", "sgasm")
    a.url_to_file = "https://soundgasm.net/sounds/176d98d189da48bbdeb6356841367172be06aa64.m4a"
    a.filename_local = "testname123333"
#     c.execute("SELECT * FROM Downloads WHERE url = ?", (a.url_to_file,))
#     pre = c.fetchone();print(pre)
    #c.execute("UPDATE Downloads SET url_sg = NULL, local_filename = NULL WHERE url = ?", (a.url_to_file,))
    #set_missing_values_df_sql(c, a)  # executescript keeps commiting my changes
    #set_missing_values_df(c, a)
    #set_missing_values_df_sqlrow(c,a)
    c.execute("SELECT url_sg, local_filename FROM Downloads WHERE url = ?", (a.url_to_file,))
    print(c.fetchone())
finally:  # prevent from commiting, whether success or an exception gets raised
    conn.rollback()
    c.execute("REPLACE INTO Downloads VALUES (359, '07.05.2017', '02:39:37', NULL, '[F4A] Take Care - Beach House cover [singing][badly playing ukuele].m4a', '[F4A] Take Care - Beach House cover [singing][badly playing ukuele]', 'https://soundgasm.net/sounds/176d98d189da48bbdeb6356841367172be06aa64.m4a', 'https://soundgasm.net/u/belle_in_the_woods/F4A-Take-Care-Beach-House-cover-singingbadly-playing-ukuele', NULL, NULL, NULL, NULL, NULL, 'Belle_in_the_woods', 'belle_in_the_woods', NULL)")
    conn.commit()
    conn.close()

('https://soundgasm.net/u/belle_in_the_woods/F4A-Take-Care-Beach-House-cover-singingbadly-playing-ukuele', '[F4A] Take Care - Beach House cover [singing][badly playing ukuele].m4a')


In [94]:
conn = sqlite3.connect("N:\\_archive\\test/trans\\soundgasmNET\\_dev\\gwarip_db.sqlite")
c = conn.cursor()
a = gw.AudioDownload("https://soundgasm.net/u/Testing1231234/testest", "sgasm")
a.url_to_file = "https://soundgasm.net/sounds/176d98d189da48bbdeb6356841367172be06aa64.m4a"
a.filename_local = "testname123333"

### Using only sql (with CASE WHEN..) is slowest by far!!! Using sqlite3.Row for dict-like access is fastest

In [95]:
%timeit set_missing_values_df_sql(c, a)

The slowest run took 52.47 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 7.78 ms per loop


In [96]:
%timeit set_missing_values_df(c, a)

1000 loops, best of 3: 314 µs per loop


In [97]:
%timeit set_missing_values_df_sqlrow(c, a)

The slowest run took 5.52 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 182 µs per loop


In [98]:
conn.close()

In [105]:
conn = sqlite3.connect("N:\\_archive\\test/trans\\soundgasmNET\\_dev\\gwarip_db.sqlite")
c = conn.cursor()
print(c.connection.row_factory)  # default for row_factory is None
c.execute("SELECT url_sg, local_filename FROM Downloads WHERE url = 'https://soundgasm.net/sounds/176d98d189da48bbdeb6356841367172be06aa64.m4a'")
print(c.fetchone())
# since this is weird and not that pracitcal ill pass the connection or connection+cursor to my fucntions
c.connection.row_factory = sqlite3.Row
c_rf = c.connection.cursor()
c_rf.execute("SELECT url_sg, local_filename FROM Downloads WHERE url = 'https://soundgasm.net/sounds/176d98d189da48bbdeb6356841367172be06aa64.m4a'")
print(c_rf.fetchone())
# if the line below is missing we get a cursor from conn that returns Row obj when fetching data
c.connection.row_factory = None
c.execute("SELECT url_sg, local_filename FROM Downloads WHERE url = 'https://soundgasm.net/sounds/176d98d189da48bbdeb6356841367172be06aa64.m4a'")
print(c.fetchone())
ct = c.connection.cursor()
ct.execute("SELECT url_sg, local_filename FROM Downloads WHERE url = 'https://soundgasm.net/sounds/176d98d189da48bbdeb6356841367172be06aa64.m4a'")
print(ct.fetchone())

None
('https://soundgasm.net/u/belle_in_the_woods/F4A-Take-Care-Beach-House-cover-singingbadly-playing-ukuele', '[F4A] Take Care - Beach House cover [singing][badly playing ukuele].m4a')
('https://soundgasm.net/u/belle_in_the_woods/F4A-Take-Care-Beach-House-cover-singingbadly-playing-ukuele', '[F4A] Take Care - Beach House cover [singing][badly playing ukuele].m4a')
('https://soundgasm.net/u/belle_in_the_woods/F4A-Take-Care-Beach-House-cover-singingbadly-playing-ukuele', '[F4A] Take Care - Beach House cover [singing][badly playing ukuele].m4a')


## Testing timeit for checking if one url is in db

In [10]:
import sqlite3
con = sqlite3.connect("N:\_archive\\test\\trans\soundgasmNET\gwarip_db.sqlite")
def check_direct_url_for_dl(db_con, direct_url):
    c = db_con.execute("SELECT url_file FROM Downloads")
    # converting to set would take just as long (for ~10k entries) as searching for it in list
    # returned as list of 1-tuples, use generator to unpack, so when we find direct_url b4
    # the last row we dont have to generate the remaining tuples and we only use it once
    file_urls = (tup[0] for tup in c.fetchall())
    if direct_url in file_urls:
        return True
    else:
        return False
    
def check_direct_url_for_dl_l(db_con, direct_url):
    c = db_con.execute("SELECT url_file FROM Downloads")
    # converting to set would take just as long (for ~10k entries) as searching for it in list
    # returned as list of 1-tuples, use generator to unpack, so when we find direct_url b4
    # the last row we dont have to generate the remaining tuples and we only use it once
    file_urls =  [tup[0] for tup in c.fetchall()]
    if direct_url in file_urls:
        return True
    else:
        return False

In [8]:
try:
    %timeit check_direct_url_for_dl(con, "https://soundgasm.net/sounds/92f9569c1ab0c0f21f709d84dce5325aad9aad4b.m4a")
finally:
    con.close()

10 loops, best of 3: 42.4 ms per loop


In [11]:
try:
    %timeit check_direct_url_for_dl_l(con, "https://soundgasm.net/sounds/92f9569c1ab0c0f21f709d84dce5325aad9aad4b.m4a")
finally:
    con.close()
# -> generator not MINIMALLY faster

10 loops, best of 3: 44.5 ms per loop


### Testing checking if url contained with dict, vs writing them out

In [20]:
supported_hosts = {
                "sgasm": "soundgasm.net",
                "chirb.it": "chirb.it/",
                "eraudica": "eraudica.com/"
            }

def check_p_url():
    found_urls = []
    for url in ["https://chiasasrb.it/u/Testing1231234/testest", "https://chirb.it/u/Testing1231234/testest"]:
        sub_url = url
        # TODO Refactor only selftext search makes sense in sep func, url check mb with loop and dict, then /gwa remove in _set_eraudica..
        if "soundgasm.net" in sub_url:
            found_urls.append(("sgasm", sub_url))
        elif "chirb.it/" in sub_url:
            found_urls.append(("chirb.it", sub_url))
        elif "eraudica.com/" in sub_url:
            found_urls.append(("eraudica", sub_url))
        
def check_p_url_dict(supported_hosts):
    found_urls = []
    for url in ["https://chiasasrb.it/u/Testing1231234/testest", "https://chirb.it/u/Testing1231234/testest"]:
        sub_url = url
    
        for host, search_for in supported_hosts.items():
            if search_for in sub_url:
                found_urls.append((host, sub_url))
                continue

In [21]:
%timeit check_p_url()
%timeit check_p_url_dict(supported_hosts)

1000000 loops, best of 3: 1.58 µs per loop
100000 loops, best of 3: 2.7 µs per loop


=> Dict method takes almost twice as long but absolute difference is only 500ns

# Testing timeits of searching for parts of html (splits vs regex vs bs4)

In [12]:
%timeit url.split("/u/", 1)[1].split("/", 1)[0]

The slowest run took 4.19 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.63 µs per loop


In [13]:
%timeit re.search("/u/(.+)/", url).group(1)

100000 loops, best of 3: 3.28 µs per loop


In [14]:
usr = re.compile("/u/(.+)/")

In [15]:
%timeit re.search(usr, url).group(1)

100000 loops, best of 3: 3.92 µs per loop


In [87]:
def sgasm_inf_split(url):
    site = urllib.request.urlopen(url)
    html = site.read().decode('utf-8')
    site.close()
    nhtml = html.split("aria-label=\"title\">")
    title = nhtml[1].split("</div>", 1)[0]
    descript = nhtml[1].split("<div class=\"jp-description\">\n          <p style=\"white-space: pre-wrap;\">")[1].split("</p>\r\n", 1)[0]
    urlm4a = nhtml[1].split("m4a: \"")[1].split("\"\r\n", 1)[0]
    

def sgasm_re(url):
    site = urllib.request.urlopen(url)
    html = site.read().decode('utf-8')
    site.close()
    
    title = re.search("class=\"jp-title\" aria-label=\"title\">(.+)<\/div>", html).group(1)
    descript = re.search("class=\"jp-description\">\s+<p.+\">(.+)</p>", html).group(1)
    urlm4a = re.search("m4a: \"(.+)\"", html).group(1)
    
    
t_re = re.compile("class=\"jp-title\" aria-label=\"title\">(.+)<\/div>")
d_re = re.compile("class=\"jp-description\">\s+<p.+\">(.+)</p>")
url_re = re.compile("m4a: \"(.+)\"")


def sgasm_re_comp(url):
    site = urllib.request.urlopen(url)
    html = site.read().decode('utf-8')
    site.close()
    
    title = re.search(t_re, html).group(1)
    descript = re.search(d_re, html).group(1)
    urlm4a = re.search(url_re, html).group(1)
    
def sgasm_bs4(url):
    site = urllib.request.urlopen(url)
    html = site.read().decode('utf-8')
    site.close()
    soup = bs4.BeautifulSoup(html, "html.parser")
    
    title = soup.select_one("div.jp-title").text
    descr = soup.select_one("div.jp-description > p").text
    urlm4a = re.search("m4a: \"(.+)\"", html).group(1)

In [33]:
%timeit sgasm_inf_split("file:///N:/_archive/test/trans/soundgasmNET/_dev/_sgasm-repo/tests/test_res/soundgasm.net_file-site.html")

1000 loops, best of 3: 427 µs per loop


In [34]:
%timeit sgasm_re("file:///N:/_archive/test/trans/soundgasmNET/_dev/_sgasm-repo/tests/test_res/soundgasm.net_file-site.html")

The slowest run took 8.02 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 436 µs per loop


In [81]:
%timeit sgasm_re_comp("file:///N:/_archive/test/trans/soundgasmNET/_dev/_sgasm-repo/tests/test_res/soundgasm.net_file-site.html")

1000 loops, best of 3: 417 µs per loop


In [88]:
%timeit sgasm_bs4("file:///N:/_archive/test/trans/soundgasmNET/_dev/_sgasm-repo/tests/test_res/soundgasm.net_file-site.html")

100 loops, best of 3: 5.71 ms per loop


In [75]:
def rip_usr_links(sgasm_usr_url):
    site = urllib.request.urlopen(sgasm_usr_url)
    html = site.read().decode('utf-8')
    site.close()
    # links zu den einzelnen posts isolieren
    nhtml = html.split("<div class=\"sound-details\"><a href=\"")
    del nhtml[0]
    user_files = []
    for splits in nhtml:
        # teil str in form von https://soundgasm.net/u/USERNAME/link-to-post> an ">" und schreibt
        # den ersten teil in die variable url
        url = splits.split("\">", 1)[0]
        # url in die liste anfuegen
        user_files.append(url)
    return user_files


def rip_usr_links_re(sgasm_usr_url):
    site = urllib.request.urlopen(sgasm_usr_url)
    html = site.read().decode('utf-8')
    site.close()
    # links zu den einzelnen posts isolieren
    
    # .+ match one or more of any char except newline, .+? -> ?: match as few as possible
    # findall returns list of strings or list of groups -> more than 1 grp -> tuples 
    # matched = re.findall("<div class=\"sound-details\"><a href=\"(.+?)\">(.+?)<\/a>", html)
    urls = re.findall("<div class=\"sound-details\"><a href=\"(.+?)\">", html)
    # unpack list of tuples
    # url, titles = zip(*matched)
    #print(url)
    return urls


def rip_usr_links_bs4(u_url):
    site = urllib.request.urlopen(u_url)
    html = site.read().decode('utf-8')
    site.close()
    soup = bs4.BeautifulSoup(html, 'html.parser')
    
    anchs = soup.select("div.sound-details > a")
    urls = [a["href"] for a in anchs]
    return urls
        

In [52]:
%timeit rip_usr_links("file:///N:/_archive/test/trans/soundgasmNET/_dev/_sgasm-repo/tests/test_res/soundgasm.net_user-site.html")

1000 loops, best of 3: 874 µs per loop


In [53]:
%timeit rip_usr_links_re("file:///N:/_archive/test/trans/soundgasmNET/_dev/_sgasm-repo/tests/test_res/soundgasm.net_user-site.html")

1000 loops, best of 3: 1.49 ms per loop


In [77]:
%timeit rip_usr_links_bs4("file:///N:/_archive/test/trans/soundgasmNET/_dev/_sgasm-repo/tests/test_res/soundgasm.net_user-site.html")

10 loops, best of 3: 84.3 ms per loop
